In [1]:
import sys
sys.path.append(r'C:\Users\bakwe\Documents\GitHub\Git extra\HW3\HW3')
# Import packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import os
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from scipy import stats
from sklearn.metrics import roc_auc_score
import HW3
from HW3.features.data_loading import loaddata
from HW3.features.data_cleaning import process_data
from HW3.features.splitdata import splitdata
from HW3.features.model import model


In [2]:
# Load data
df = loaddata(r'C:\Users\bakwe\Documents\GitHub\Git extra\HW3\sample_diabetes_mellitus_data.csv')
print(df.head())

   Unnamed: 0  encounter_id  hospital_id   age        bmi  elective_surgery  \
0           0        214826          118  68.0  22.732803                 0   
1           1        246060           81  77.0  27.421875                 0   
2           2        276985          118  25.0  31.952749                 0   
3           3        262220          118  81.0  22.635548                 1   
4           4        201746           33  19.0        NaN                 0   

   ethnicity gender  height hospital_admit_source  ... ventilated_apache  \
0  Caucasian      M   180.3                 Floor  ...                 0   
1  Caucasian      F   160.0                 Floor  ...                 1   
2  Caucasian      F   172.7  Emergency Department  ...                 0   
3  Caucasian      F   165.1        Operating Room  ...                 1   
4  Caucasian      M   188.0                   NaN  ...                 0   

   wbc_apache aids cirrhosis  hepatic_failure  immunosuppression  le

In [3]:
# Clean data
columns_drop = ['age', 'gender', 'ethnicity']
columns_fill = ['height', 'weight']
columns_hot = ['ethnicity']
columns_binary = 'gender'
df_processed = process_data(df, columns_drop, columns_fill, columns_hot, columns_binary)

print(df_processed.shape)
print(df_processed.info())

(9368, 60)
<class 'pandas.core.frame.DataFrame'>
Index: 9368 entries, 0 to 9999
Data columns (total 60 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   Unnamed: 0                   9368 non-null   int64  
 1   encounter_id                 9368 non-null   int64  
 2   hospital_id                  9368 non-null   int64  
 3   age                          9368 non-null   float64
 4   bmi                          7746 non-null   float64
 5   elective_surgery             9368 non-null   int64  
 6   ethnicity                    9368 non-null   object 
 7   gender                       9368 non-null   object 
 8   height                       9368 non-null   float64
 9   hospital_admit_source        8518 non-null   object 
 10  icu_admit_source             9347 non-null   object 
 11  icu_id                       9368 non-null   int64  
 12  icu_stay_type                9368 non-null   object 
 13  icu_type    

C:\Users\bakwe\Documents\GitHub\Git extra\HW3\HW3\HW3\features\data_cleaning.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean[columns_fill] = df_clean[columns_fill].apply(lambda col: col.fillna(col.mean()))
C:\Users\bakwe\Documents\GitHub\Git extra\HW3\HW3\HW3\features\data_cleaning.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_clean['Binary'] = df_clean[columns_binary].apply(lambda x: 1 if x == 'M' else 0)


In [4]:
# Function to check the results of cleaning (additional function, not required in the HW)
def missing_values_table(df):
        # Total missing values
        mis_val = df.isnull().sum()
        
        # Percentage of missing values
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        
        # Make a table with the results
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        
        # Rename the columns
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        
        # Sort the table by percentage of missing descending
        mis_val_table_ren_columns = mis_val_table_ren_columns[
            mis_val_table_ren_columns.iloc[:,1] != 0].sort_values(
        '% of Total Values', ascending=False).round(1)

                # Print some summary information
        print ("Your selected dataframe has " + str(df.shape[1]) + " columns.\n"      
            "There are " + str(mis_val_table_ren_columns.shape[0]) +
              " columns that have missing values.")
        
        # Return the dataframe with missing information
        return mis_val_table_ren_columns

In [5]:
# Check the results of cleaning for the columns of interest
columns_to_check = ['age', 'gender', 'ethnicity', 'height', 'weight']
result = missing_values_table(df_processed[columns_to_check])
print(result)

Your selected dataframe has 5 columns.
There are 0 columns that have missing values.
Empty DataFrame
Columns: [Missing Values, % of Total Values]
Index: []


In [6]:
X_train, X_test, y_train, y_test = splitdata(df_processed, 'diabetes_mellitus')

# Print the values
print(f'X_train: {X_train}')
print(f'X_test: {X_test}')
print(f'y_train: {y_train}')
print(f'y_test: {y_test}')

X_train:       Unnamed: 0  encounter_id  hospital_id   age        bmi  \
952          952        160030          149  50.0        NaN   
4283        4283        235736           81  63.0  31.835347   
5521        5521        242191          118  63.0  25.062103   
9655        9655        191087           89  79.0  25.015328   
3549        3549        271596          118  31.0  19.228087   
...          ...           ...          ...   ...        ...   
6062        6062        201939          118  59.0        NaN   
5494        5494        227860           31  67.0  18.301839   
5703        5703        190499          118  85.0  18.296026   
913          913        165385          118  43.0  23.661272   
7681        7681        242945          118  23.0  22.496401   

      elective_surgery         ethnicity gender  height hospital_admit_source  \
952                  0         Caucasian      F  170.20  Emergency Department   
4283                 1          Hispanic      M  195.59     

In [7]:
# Execute training and metrics calculations
features = ['age', 'height', 'weight', 'aids', 'cirrhosis', 'hepatic_failure',
            'immunosuppression', 'leukemia', 'lymphoma', 'solid_tumor_with_metastasis']
target = ['diabetes_mellitus']  
X_train = X_train[features] 
X_test = X_test[features]
y_train = y_train
y_test = y_test
result = model(X_train, X_test, y_train, y_test)
print(result)

Train ROC AUC: 0.6703545314832863
Test ROC AUC: 0.6689398850455878
(0.6703545314832863, 0.6689398850455878)
